In [4]:
import tensorflow as tf
from tensorflow import keras
import os
from os.path import isfile, join
import numpy as np
import shutil
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [ ]:
data_directory = '16000_pcm_speeches'
audito_folder = 'audio'
noise_folder = 'noise'

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, nosie_fodler)

In [ ]:
audio_path

In [ ]:
noise_path

In [ ]:
voicefile_names=os.listdir('1600_pcm_speeches/audio')

In [ ]:
voicefile_names

In [ ]:
noisefile_names = os.listdir('1600_pcm_speeches/noise')
noisefile_names

In [ ]:
voice_files_count=[]

In [ ]:
for t in voicefile_names:
    p=os.listdir('1600_pcm_speeches/audio'+'/'+t)
    len_class=len(p)
    voice_files_count.append(len_class)

In [ ]:
voice_files_count

In [ ]:
noise_files_count=[]

In [ ]:
for t in voicefile_names:
    p=os.listdir('1600_pcm_speeches/noise'+'/'+t)
    len_class=len(p)
    voice_files_count.append(len_class)

In [ ]:
noise_files_count

In [ ]:
import plotly.express as px
import numpy

random_x=voicefile_names
random_y=voice_files_count

fig=px.bar(random_x,y=random_y)
fig.show() # bars show balanced voice samples for each person

In [ ]:
import plotly.express as px
import numpy

random_x=noisefile_name
random_y=noise_files_count

fig=px.bar(random_x,y=random_y)
fig.show() # bars show balanced voice samples for each person
# only two noise files but each of them are very lengthy

In [ ]:
valid_split=0.1 # train validation split
shuffle_seed=43 # shuffle data
sample_rate=16000
scale=0.5 # amplitude
batch_size=128 # depends on computer
epochs=15

In [ ]:
# get list of all noise files
noise_paths=[]
for subdir in os.listdir(noise_path):
    subdir_path=Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths+= [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endwith('.wav')
        ]

In [ ]:
noise_paths

In [ ]:
import librosa
x,sr=librosa.load('1600_pcm_speeches/noise/other/exercise_bike.wav')

In [ ]:
print(x.shape)
print(sr) # normally 22050, we need to convert to 16000

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
plt.figure(size=14,5))
librosa.display.waveshow(x,sr=sr)

In [ ]:
X=librosa.stft(x)
Xdb=librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14,5))
librosa.display.sepcshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

In [ ]:
import IPython.display as ipd
ipd.Audio('1600_pcm_speeches/noise/other/exercise_bike.wav')

### split noise into chunks of 16,000 steps each

In [ ]:
#os.system(command)
# cut noise samples into smaller samples
def load_noise_sample(path):
    sample,sampling_rate=tf.audio.decode_wav(tf.io.read_file(path),desired_channels=1)
    print('sampling rate of original audio',sampling_rate)
    if sampling_rate = sample_rate:
        print('shape',sample.shape[0])
        slices=int(sample.shape[0]/sample_rate)
        print(slices)
        sample=tf.split(sample[:slices*sample_rate],slices)
    else:
        print('sampling rate for', path, 'is incorrect')
        return None
    
noises=[]
for path in noise_paths:
    sample=load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises=tf.stack(noises)        

In [ ]:
noises

### dataset generation

In [ ]:
def paths_and_labels_to_dataset(audio_paths,labels):
    path_ds=tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds=path_ds.map(lambdax:path_to_audio(x))
    label_ds=tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip(audio_ds,label_df))

In [ ]:
def path_to_audio(path):
    audio=tf.io.read_file(path)
    audio,_=tf.audio.decode_wav(audio,1,sample_rate)
    return audio

### add noise to dataset

In [ ]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd=tf.random.uniform((tf.shape[0],),0.noises.shape[0],dtype=tf.int32)
        noise=tf.gather(noises,tf_rnd,axis=0)
        prop=tf.math.reduce_max(audio,axis=1)/tf.math.reduce_max(noise,axis=1)
        prop=ft.repeat(tf.expand_dims(prop,axis=1),tf/shape(audio)[1],axis=1)
        audio=audio+noise*prop*scale
    return audio

In [ ]:
def audio_to_fft(audio):
    audio=tf.squeeze(audio,axis=-1)
    fft=tf.signal.fft(tf.cast(tf.complex(real=audio,imag=tf.zeros_like(audio)),tf.complex64))
    fft=tf.expand_dims(fft,axis=-1)
    return tf.math.abs(fft[:,:(audio.shape[1]//2),:])


In [ ]:
class_names=os.listdir(audio_path)
print(class_names,)

audio_paths=[]
labels=[]
for label,name in enumerate(class_names):
    print('speaker:', (name))
    dir_path=Path(audio_path)/name
    speaker_sample_paths=[os.path.join(dir_path,filepath) for filepath in os.listdir(dir_path) if filepath.endswith('.wav')]
    audio_paths+=speaker_sample_paths
    labels+=[label]*len(spearker_sample_paths)

In [ ]:
#shuffle to generate random data
rng=np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng=np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [ ]:
#split into training and validation
num_val_samples=int(valid_split*len(audio_paths))
train_audio_paths=audio_paths[:-num_val_samples]
train_labels=labels[:-num_val_samples]

valid_audio_paths=audio_paths[-num_val_sample:]
valid_labels=labels[-num_val_samples:]

In [ ]:
#create datasets, one for trainig and the other for validation
train_ds=paths_and_labels_to_dataset(train_audio_paths,train_labels)
train_ds=train_ds.shuffle(buffer_size=batch_size*8, seed=shuffle_seed).batch(batch_size)

valid_ds=paths_and_labes_to_dataset(valid_audio_paths,valid_labels)
valid_ds=valid_ds.shuffle(buffer_size=32*8, seed=shuffle_seed).batch(32)

### feature extraction

In [ ]:
# add noise tot the training set
train_ds=train_ds.map(lambda x,y:(add_noise(x,noises,scale=scale),y),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE,)

#transform audio wave to the frequency domain using 'audio_to_fft'
train_ds=train_ds.map(lambda x,y: (audio_to_fft(x),y), num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_ds=train_ds.prefetch(ft.data.experimental.AUTOTUNE)

valid_ds=valid_ds.map(lambda x,y: (audio_to_fft(x),y),num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_ds=valid_ds.prefetch(tf.data.experimental.AUTOTUNE)

### model

In [ ]:
from tensorflow.keras.layers import Conv1D

In [ ]:
def residual_block(x, filters, conv_num=3,activation='relu'):
        s=keras.layers.Conv1D(filters,1,padding='same')(x)
        
        for i in range(conv_num-1):
            x=keras.layers.Conv1D(filters,3,padding='same')(x)
            x=keras.layers.Activation(activation)(X)
            
        x = keras.layers.Conv1D(filters,3,padding='same')(x)
        x=keras.layers.Add()([x,s])
        x=keras.layers.Activation(activation)(x)
        
        return keras.layers.MaxPool1D(pool_size=2,strides=2)(x)

def build_model(input_shape,num_classes):
    inputs=keras.layers.Input(shape=input_shape,name='input')
    
    x=residual_block(inputs,16,2)
    x=residual_block(inputs,32,2)
    x=residual_block(inputs,64,3)
    x=residual_block(inputs,128,3)
    x=residual_block(inputs,128,3)
    x=keras.layers.AveragePooling1D(pool_size=3,strides=3)(x)
    x=keras.layers.Flatten()(x)
    x=keras.layers.Dense(256,activation='relu')(x)
    x=keras.layers.Dense(128,activation='relu')(x)
    
    outputs=keras.layers.Dense(num_class,activation='softmax', name='output')(x)
    return keras.model.Model(inputs=inputs,outputs=outputs)

model.build_model((sample_rate//2,1),len(class_names))
model.summary()
model.compile(optiizer='Adam',loss='sparse_categorical_crossentropy', metrics-['accuracy'])
model_save_filename='model.h5'
earlystipping_cb=keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb=keras.callbacks.ModelCheckpoint(model_save_filename,monitor='val_accuracy',save_best_only=True)


In [ ]:
epochs=1

### training

In [ ]:
history=model.fit(
    trian_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

### accuracy

In [ ]:
print('accuracy of model:', model.evaluate(valid_ds)) # loss: 0.0914; accuracy: 0.9800

### predict

In [ ]:
SAMPLES_TO_DISPLAY=10

test_ds=paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds=test_ds.shuffle(buffer_size=batch_size*8, seed=shuffle_seed).batch(batch_size)

test_ds=test_ds.map(lambda x,y: (add_noise(x, noises, scale=scale),y))

for audios,labels in test_ds.take(1):
    ffts=audio_to_fit(audios)
    y_pred=model.predict(ffts)
    rnd=np.random.randint(0,batch_size,SAMPLES_TO_DISPLAY)
    audios=audios.numpy()[rnd,:,:]
    labels=labels.numpy()[rnd]
    y_pred=np.argmax(y_pred,axis=-1)[rnd]
    
    for index in range(SAMPLES_TO_DISPLAY):
        print('speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m'.format(
            "[92m" if labels[index] == y_pred[index] else "[91m",
            class_names[labels[index]],
            "[92m" if labels[index] == y_pre[index] else "[91m",
            class_names[y_pred[index]],
        )
             )
        if labels[index] == y_pred[index]:
            print('welcome')
        else:
            print('sorry')
        print("The speaker is" if labels[index] == y_pred[iondex] else "", class_names[y_pred[index]]) ######################
                    

In [ ]:
'''
spkear: Julia_Gillard Predicted: Julia_gillard
welcome
'''

### predict the speaker from test dataset for real time period

In [ ]:
def paths_to_dataset(audio_paths):
    path_ds=tf.data.Dataset.from_tensor_slices(audio_paths)
    return tf.data.Dataset.zip((path_ds))

def predict(path,labels):
    test = paths_and_labels_to_dataset(path,labels)
    
    test=test.shuffle(buffer_size=batch_size*8,seed=shuffle_seed).batch(batch_size)
    test=test.prefetch(ft.data.experimental.AUTOTUNE)
    
    test=test.map(lambda x,y: (add_noise(x,noises,scale=scale),y))
    
    for audios, labels in test.take(1):
        ffts=audio_to_fft(audios)
        y_pred=model.predict(ffts)
        rnd=np.random.randint(0,1,1)
        audios=audios.numpy[rnd,;]
        labels=labels.numpy[rnd]
        y_pred=np.argmax(y_pred, axis=-1)[rnd]
        
    for index in range(1):
        print(
            "speaker:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m",y_pred[index],
                "[92m",y_predi[index])
        )
        print("speaker predicted:", class_names[y_pred[index]])

In [ ]:
path=['1600_pcm_speeches/audio/Jens_Stoltenberg/1013.wav']
labels=['unknow']
try:
    predict(path,labels)
except:
    print('Error! check if the file is correctly passed or not!')